In [1]:
from netCDF4 import Dataset
import wrf
import xarray as xr
import numpy as np

import cartopy.crs as crs


from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cartopy.feature as cfe

import pandas as pd


import matplotlib.pyplot as plt



from cartopy.feature import NaturalEarthFeature

import cartopy.io.shapereader as shpr

import cartopy.io.img_tiles as cimgt

from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)

from matplotlib.cm import get_cmap
from wrf import getvar, interplevel, to_np, get_basemap, latlon_coords
import imageio

ModuleNotFoundError: No module named 'netCDF4'

In [2]:
stamen_terrain = cimgt.Stamen('terrain-background')

states = NaturalEarthFeature(category="cultural", scale="10m",
                                 facecolor="none",
                                 name="admin_1_states_provinces_shp")
roads = NaturalEarthFeature(category='cultural',
                                     name='roads',
                                     scale='10m',
                                     facecolor='none')

NameError: name 'cimgt' is not defined

# Dirección y velocidad del viento
## Función para hacer las animaciones de la dirección y velocidad del viento

In [3]:
# Abrir el dataset
def gif_viento(dominio):
    ncfile = Dataset('wrfout_d{}_2021-05-11_00:00:00'.format(dominio))

    for i in range(0, 169):

        ua = getvar(ncfile, "uvmet10", units="kt",timeidx=i)[0,:]
        va = getvar(ncfile, "uvmet10", units="kt",timeidx=i)[1,:]
        wspd = getvar(ncfile, "wspd_wdir10", units="kt",timeidx=i)[0,:]



        # Get the lat/lon coordinates
        lats, lons = latlon_coords(wspd)

        # Get the map projection information
        cart_proj = get_cartopy(wspd)

        # Create the figure
        fig = plt.figure(figsize=(12,9))
        ax = plt.axes(projection=cart_proj)


        # Add the wind speed contours
        levels = [j for j in range(0,50, 6)]
        wspd_contours = plt.contourf(to_np(lons), to_np(lats), to_np(wspd),
                                     levels=levels,
                                     cmap=get_cmap("jet"),
                                     transform=crs.PlateCarree())
        plt.colorbar( wspd_contours,ax=ax, orientation="horizontal", pad=.1)

        # Add the 500 hPa wind barbs, only plotting every 125th data point.
        plt.barbs(to_np(lons[::3,::3]), to_np(lats[::3,::3]),to_np(ua[::3, ::3]), to_np(va[::3, ::3]),transform=crs.PlateCarree(), length=4)

        # Set the map bounds
        ax.set_xlim(cartopy_xlim(wspd))
        ax.set_ylim(cartopy_ylim(wspd))

        ax.add_feature(roads, edgecolor="black" ,linewidth=0.5)
        ax.add_feature(states, linewidth=.7, edgecolor="black")

        ax.gridlines()

        ax.add_feature(states, linewidth=0.5, edgecolor="black")
        ax.coastlines('50m', linewidth=0.8)
        
        dia = int(i*3/24)
        hora =i*3 - dia*24
        
        plt.title("10 M de Altura, Velocidad del viento (kt), Barras de viento (kt) \n Dia {} Hora {}:00:00".format(dia+11,hora))

        plt.savefig('figs/d{}/viento/d{}_viento_{}.png'.format(dominio,dominio,i))
        plt.close(fig)


    with imageio.get_writer('figs/d{}/d{}_viento.gif'.format(dominio,dominio), mode='I') as writer:
        for i in range(0,169):
            filename = 'figs/d{}/viento/d{}_viento_{}.png'.format(dominio,dominio,i)
            image = imageio.imread(filename)
            writer.append_data(image)

## Dominio 1

In [4]:
gif_viento("01")

<img src="figs/d01/d01_viento.gif" />

## Dominio 2

In [10]:
gif_viento("02")

<img src="figs/d02/d02_viento.gif" />

## Dominio 3


In [11]:
gif_viento("03")

<img src="figs/d03/d03_viento.gif" />